In [1]:
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import GradientBoostingRegressor
import math
from scipy.optimize import minimize
import scipy.sparse as sp
from scipy.sparse import csr_matrix
from scipy.sparse import vstack
from scipy.sparse import hstack
from sklearn.model_selection import GridSearchCV
from scipy.optimize import curve_fit
import seaborn as sns
from scipy import stats
from scipy.stats import poisson
from scipy.special import erf
from scipy.stats import gamma
from sklearn.model_selection import RandomizedSearchCV
import time
from scipy.optimize import newton
from scipy.stats import norm
from itertools import product
import nbimporter
import functions
from tqdm import tqdm

In [2]:
pi0_event_info = #csv
pi0_layer_info = #csv

pi0_event_info = pi0_event_info.astype('float32')
pi0_layer_info = pi0_layer_info.astype('float32')


psb = pi0_layer_info.iloc[:, 1:64] #9 eta * 7 phi
emb1 = pi0_layer_info.iloc[:, 64:65+50] #17 eta * 3 phi
emb2 = pi0_layer_info.iloc[:, 65+50:65+51+62] #9 eta * 7 phi
emb3 = pi0_layer_info.iloc[:, 65+51+62:65+51+62+63] #9 eta * 7 phi
hab1 = pi0_layer_info.iloc[:,65+51+62+63:] #9 eta * 7 phi

In [3]:
def sum_and_restructure(df, df_name, num_segments, segment_size):
    #Convert the DataFrame to a NumPy array for manipulation
    data_array = df.to_numpy()
    
    #Reshape the array to 3D with dimensions: [num_rows, segment_size, num_segments]
    #This groups each set of segments together for each row
    reshaped_array = data_array.reshape(-1, num_segments, segment_size)
    
    #Sum over the last axis (num_segments) to combine the segments
    summed_array = reshaped_array.sum(axis=1)
    
    #Convert the summed array back into a DataFrame with appropriate column names
    column_names = [f"{df_name} Et{i+1}" for i in range(segment_size)]
    summed_df = pd.DataFrame(summed_array, columns=column_names)
    
    return summed_df

In [4]:
psb_Et = sum_and_restructure(psb, "psb", 7, 9)
emb1_Et = sum_and_restructure(emb1, "emb1", 3, 17)
emb2_Et = sum_and_restructure(emb2, "emb2", 7, 9)
emb3_Et = sum_and_restructure(emb3, "emb3", 7, 9)
hab1_Et = sum_and_restructure(hab1, "hab1", 7, 9)

In [5]:
psb_eta = pi0_event_info["psb eta"]
emb1_eta = pi0_event_info["emb1 eta"]
emb2_eta = pi0_event_info["emb2 eta"]
emb3_eta = pi0_event_info["emb3 eta"]
hab1_eta = pi0_event_info["hab1 eta"]
psb_emb1 = pd.DataFrame({"psb - emb1": (psb_eta - emb1_eta)})
emb1_emb2 = pd.DataFrame({"emb1 - emb2": (emb1_eta - emb2_eta)})
emb2_emb3 = pd.DataFrame({"emb2 - emb3": (emb2_eta - emb3_eta)})
emb3_hab1 = pd.DataFrame({"emb3 - hab1": (emb3_eta - hab1_eta)})

In [6]:
Et_total = []
for index, row in pi0_layer_info.iterrows():
    Et = row[1:].to_numpy() 
    Et_total.append(np.sum(Et))

Et_total = pd.DataFrame({"Et total": Et_total})

In [7]:
feature_set = pd.concat([psb_eta,
                        emb1_eta, 
                        emb2_eta,
                        emb3_eta,
                        hab1_eta,
                        psb_emb1,
                        emb1_emb2,
                        emb2_emb3,
                        emb3_hab1,
                        psb_Et, emb1_Et, emb2_Et, emb3_Et, hab1_Et], axis = 1)

In [8]:
all_features_p_et = pd.concat([feature_set, Et_total], axis = 1)

In [9]:
label = pi0_event_info["z"]

In [10]:
all_features_train, all_features_test, label_train, label_test = train_test_split(all_features_p_et, label, test_size=0.3, random_state=30)

In [11]:
X_train = all_features_train.drop('Et total', axis=1)
X_test = all_features_test.drop('Et total', axis=1)
y_train = label_train
y_test = label_test

In [12]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

params = {
    'max_depth': 6,
    'objective': 'reg:squarederror',  
    'eval_metric': 'rmse',  
    'alpha': 0,
    'learning_rate': 0.1 

}

num_round = 600  #Number of boosting rounds
bdt_model = xgb.train(params, dtrain, num_boost_round=num_round)

predictions = bdt_model.predict(dtest)

mse = mean_squared_error(y_test, predictions)

rmse = np.sqrt(mse)